## Classification (b)

In [1]:
from neuralnet import *
from random import shuffle
import numpy as np

# Load the configuration.
config = load_config("./data")

# Create the model
#model  = Neuralnetwork(config)
# Load the data
#x_train, y_train = load_data(path="./data", mode="train") # output normalized and one hot encoding
#x_test, y_test = load_data(path="./data", mode="test")# output normalized and one hot encoding

# Use 10 examples one from each category

In [2]:

# Data loading and spliting
x_b, y_b = load_data(path="./data", mode="train")

xtrain = []
ytrain = []

p = 0;
for k in range(5):
    xtrain.extend(x_b[p :p+10])
    ytrain.extend(y_b[p :p+10])
    p += 10000
        
xtrain = np.array(xtrain)
ytrain = np.array(ytrain)


In [3]:
# Load model para
config_prob_b = {}
config_prob_b['layer_specs'] = [3072, 50, 50, 10]
config_prob_b['activation'] = 'ReLU'
config_prob_b['learning_rate'] = 0.15 
config_prob_b['batch_size'] = 128 
config_prob_b['epochs'] = 100  
config_prob_b['early_stop'] = True 
config_prob_b['early_stop_epoch'] = 5  
config_prob_b['L2_penalty'] = 0  
config_prob_b['momentum'] = True  
config_prob_b['momentum_gamma'] = 0.9  

model_b = Neuralnetwork(config_prob_b)
model_b.forward(xtrain,ytrain)
model_b.backward()


array([[ 2.73889574e+001, -5.30624852e+001, -2.31885315e+001, ...,
        -1.27680901e+001, -6.89952795e+001,  2.75510320e+001],
       [-9.54164444e+000,  1.59587202e+001, -1.20804399e+001, ...,
         1.18632812e+001, -1.44246347e+001, -3.34259769e+001],
       [ 3.40306688e+001, -2.25775479e+001,  3.82638783e+000, ...,
        -1.61362236e+001, -7.81379976e+000,  1.50836274e+001],
       ...,
       [ 2.52036162e+001,  2.54091176e+001, -4.73831886e+001, ...,
         6.36402252e+001,  1.54768732e+001,  3.55441409e+001],
       [ 4.32648190e-208,  2.86442538e-208, -3.64763869e-208, ...,
        -1.01943238e-208,  7.06325453e-208,  3.51790149e-208],
       [ 3.35966225e+001,  1.34533738e+001, -3.69758131e+001, ...,
         4.49671495e+001,  2.56739003e+001,  4.33933252e+001]])

In [4]:
# d_b d_w comparison

## Part (b) Estimation of bias weight and weight
def Num_Est_b(model, layer, eps, input_idx, output_idx):
    layer.b[0][output_idx] += eps # E(w+e)
    loss_1 = model.forward(xtrain, ytrain)
    layer.b[0][output_idx] -= 2*eps # E(w-e)
    loss_2 = model.forward(xtrain, ytrain)
    layer.b[0][output_idx] += eps # back to normal
    return (loss_1 - loss_2) / (2 * eps) # Numerical estimation for dEdW

def Num_Est_w(model, layer, eps, input_idx, output_idx):
    layer.w[input_idx][output_idx] += eps # E(w+e)
    loss_1 = model.forward(xtrain, ytrain)
    layer.w[input_idx][output_idx] -= 2*eps # E(w-e)
    loss_2 = model.forward(xtrain, ytrain)
    layer.w[input_idx][output_idx] += eps # back to normal
    return (loss_1 - loss_2) / (2 * eps) # Numerical estimation for dEdW

eps = 1e-2 # difference expectation to 10e-4


db = []
dw = []
db_est = []
dw_est = []
for layer in model_b.layers:
    if isinstance(layer, Activation):
        continue
    db_est.append(Num_Est_b(model_b, layer, eps, 0, 1))
    db.append(layer.d_b[1]) 
    dw_est.append([Num_Est_w(model_b, layer, eps, 0, 0), Num_Est_w(model_b,layer, eps, 0, 1)])
    dw.append([layer.d_w[0][0] ,layer.d_w[0][1]])
        
print('(db - db_est): {}'.format(np.array(db)-np.array(db_est)))
print('(dw - dw_est): {}'.format(np.array(dw)-np.array(dw_est)))

print('(db ): {}'.format(np.array(db)))
print('(db_est): {}'.format(np.array(db_est)))


(db - db_est): [-2.46360175e-01 -1.14245712e-01  4.81279231e-07]
(dw - dw_est): [[ -1.05069756  -0.97181213]
 [-15.4772594   -6.40126366]
 [-15.63377804  -5.91144036]]
(db ): [-2.46360175e-01 -1.14245712e-01  4.81279231e-07]
(db_est): [0. 0. 0.]
